In [17]:
import tensorflow as tf

def efficient_net(input_tensor, resolution, depth, width, num_classes, dropout_rate=0.2, include_top=True):
    def round_filters(filters, width_multiplier, divisor):
        filters *= width_multiplier
        new_filters = int(filters + divisor / 2) // divisor * divisor
        new_filters = max(divisor, new_filters)
        if new_filters < 0.9 * filters:
            new_filters += divisor
        return int(new_filters)

    def round_repeats(repeats, depth_multiplier):
        return int(depth_multiplier * repeats)

    def swish(x):
        return x * tf.nn.sigmoid(x)

    def fused_mbconv_block(x, expand_filters, out_filters, strides):
        y = tf.keras.layers.Conv2D(expand_filters, kernel_size=1, padding='same', use_bias=False)(x)
        y = tf.keras.layers.BatchNormalization(axis=3)(y)
        y = swish(y)

        y = tf.keras.layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False)(y)
        y = tf.keras.layers.BatchNormalization(axis=3)(y)
        y = swish(y)

        y = tf.keras.layers.Conv2D(out_filters, kernel_size=1, padding='same', use_bias=False)(y)
        y = tf.keras.layers.BatchNormalization(axis=3)(y)

        if strides == 1 and x.shape[-1] == out_filters:
            y = tf.keras.layers.Add()([x, y])

        return y

    def mbconv_block(x, expand_filters, out_filters, strides):
        y = tf.keras.layers.Conv2D(expand_filters, kernel_size=1, padding='same', use_bias=False)(x)
        y = tf.keras.layers.BatchNormalization(axis=3)(y)
        y = swish(y)

        y = tf.keras.layers.DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False)(y)
        y = tf.keras.layers.BatchNormalization(axis=3)(y)
        y = swish(y)

        y = tf.keras.layers.Conv2D(out_filters, kernel_size=1, padding='same', use_bias=False)(y)
        y = tf.keras.layers.BatchNormalization(axis=3)(y)

        if strides == 1 and x.shape[-1] == out_filters:
            y = tf.keras.layers.Add()([x, y])

        return y

    def cbam_block(x):
        channels = x.shape[-1]
        y = tf.keras.layers.GlobalAveragePooling2D()(x)
        y = tf.keras.layers.Reshape((1, 1, channels))(y)
        y = tf.keras.layers.Dense(channels // 4, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(y)
        y = tf.keras.layers.Dense(channels, activation='sigmoid', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(y)
        y = x * y
        z = tf.keras.layers.GlobalMaxPooling2D()(y)
        z = tf.keras.layers.Dense(channels // 4, activation='relu', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(z)
        z = tf.keras.layers.Dense(channels, activation='sigmoid', kernel_initializer='he_normal', use_bias=True, bias_initializer='zeros')(z)
        z = tf.keras.layers.Reshape((1, 1, channels))(z)
        z = y * z
        return z

    # calculate number of layers with fused_mbconv
    num_fused_layers = 2
    num_layers = depth

    # Calculate width multiplier for each block
    widths = [1, 6, 6, 6]
    width_multiplier = width

    # Calculate depth multiplier for each block
    depths = [1, 2, 2, 2]
    depth_multiplier = depth

    # Number of filters in each block
    filters = [32, 16, 24, 40, 80, 112, 192, 320, 1280]

    # First convolution layer
    x = tf.keras.layers.Conv2D(round_filters(filters[0], width_multiplier, 8), kernel_size=3, strides=(2, 2),
                               padding='same', use_bias=False)(input_tensor)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = swish(x)

    # Blocks
    num_layers_remaining = num_layers
    for i in range(1, len(filters) - 1):
        num_layers_block = round_repeats(num_layers_remaining, depth_multiplier)
        num_layers_remaining -= num_layers_block

        # use fused_mbconv_block for the first two blocks
        if i <= num_fused_layers:
            x = fused_mbconv_block(x, round_filters(filters[i], width_multiplier, 8),
                                   round_filters(filters[i + 1], width_multiplier, 8),
                                   strides=(1 if i == 1 else 2))
        # use mbconv_block for the remaining blocks
        else:
            x = mbconv_block(x, round_filters(filters[i], width_multiplier, 8),
                             round_filters(filters[i + 1], width_multiplier, 8),
                             strides=(1 if i == 1 else 2))

        # integrate CBAM block after every other block
        if i % 2 == 0:
            x = cbam_block(x)

    # Last convolution layer
    x = tf.keras.layers.Conv2D(round_filters(filters[-2], width_multiplier, 8), kernel_size=1, padding='same',
                               use_bias=False)(x)
    x = tf.keras.layers.BatchNormalization(axis=3)(x)
    x = swish(x)

    # Include top layers (classification head)
    if include_top:
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)
        x = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(input_tensor, x)

    return model



In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data generators for training and validation
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
resolution = 224
num_epochs = 30
batch_size = 32
learning_rate = 0.001
train_dir = '/Users/hemalatha/Downloads/datasetB/train'
test_dir = '/Users/hemalatha/Downloads/datasetB/test'

# Define model hyperparameters
input_shape = (224, 224, 3)
num_classes = 5
dropout_rate = 0.2
width = 1
depth = 1.0
batch_size = 32
epochs = 30

# Instantiate model
inputs = tf.keras.Input(shape=input_shape)
model = efficient_net(inputs, resolution=input_shape[0], depth=depth, width=width, num_classes=num_classes, dropout_rate=dropout_rate, include_top=True)

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate)
loss = 'categorical_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(resolution, resolution),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(resolution, resolution),
    batch_size=batch_size,
    class_mode='categorical')

# Train the model
model.fit(train_generator, epochs=num_epochs, batch_size=batch_size)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)

# Save the model
model.save('efficientnet_with_cbam_fuused.h5')


2023-02-28 14:07:36.316076: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 29329 images belonging to 5 classes.
Found 9164 images belonging to 5 classes.
Epoch 1/30
917/917 [==============================] - 962s 1s/step - loss: 0.6037 - accuracy: 0.7654
Epoch 2/30
917/917 [==============================] - 945s 1s/step - loss: 0.2407 - accuracy: 0.9130
Epoch 3/30
917/917 [==============================] - 956s 1s/step - loss: 0.1616 - accuracy: 0.9421
Epoch 4/30
917/917 [==============================] - 953s 1s/step - loss: 0.1275 - accuracy: 0.9552
Epoch 5/30
917/917 [==============================] - 966s 1s/step - loss: 0.1018 - accuracy: 0.9643
Epoch 6/30
917/917 [==============================] - 943s 1s/step - loss: 0.0820 - accuracy: 0.9705
Epoch 7/30
917/917 [==============================] - 952s 1s/step - loss: 0.0801 - accuracy: 0.9730
Epoch 8/30
917/917 [==============================] - 957s 1s/step - loss: 0.0575 - accuracy: 0.9804
Epoch 9/30
917/917 [==============================] - 925s 1s/step - loss: 0.0580 - accuracy: 0.9798
Epoch 

In [2]:
import keras
print(keras.__version__)


2023-04-10 21:43:54.352919: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0
